In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats import ks_2samp

In [2]:
#arquitectura neuronal
class HaloToGalaxyModel(nn.Module):
    def __init__(self, input_size=4, output_size=1, hidden_dim=64):
        super(HaloToGalaxyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, hidden_dim) 
        self.fc3 = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  
        return x

# Función para cargar datos desde un CSV
def load_data_from_csv(file_path):
    data = pd.read_csv(file_path)
    X = data.iloc[:, 5:10].values  
    #y = data.iloc[:, 12:16].values  
    y = data.iloc[:, 12].values  #esto sólo carga la masa, la gracia es que para cada modelo usar un única columa
    return X, y


class customLossYan(nn.Module):
    def __init__(self, quantiles):
        super(customLossYan, self).__init__()
        self.quantiles = quantiles

    def forward(self, y_true, y_pred):
        #y_true_one_hot = F.one_hot(y_true, num_classes=y_pred.size(1)).float()
        losses = []
        for i, q in enumerate(self.quantiles):
            errors = y_true - y_pred[:, i]
            losses.append(
                torch.max((q - 1) * errors, q * errors).unsqueeze(1)
            )
        loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
        return loss


def ks_test_metric(y_true, y_pred):    
    y_true = y_true.cpu().numpy()
    y_pred = torch.argmax(y_pred, dim=1).cpu().numpy()
    
    ks_statistic, p_value = ks_2samp(y_true, y_pred)
    return ks_statistic, p_value

In [3]:
#cargar datos
file_path = 'datasetcompleto.csv'  
X, y = load_data_from_csv(file_path)

In [4]:
k = 50 #división de bins, 50 es lo que dice el paper
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
#transformar etiquetas a bins para problema de clasificacion
bins = np.linspace(y.min(), y.max(), k )
y_binned = np.digitize(y, bins) - 1
X = torch.tensor(X, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.float32).to(device)

In [6]:
print(len(y))

174785


In [7]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.12, random_state=42)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)


hidden_dim = 100 #tamaño de las capas ocultas
num_epochs = 1000
early_stop_patience = 30
best_val_loss = float('inf')
epochs_no_improve = 0

model = HaloToGalaxyModel(X.shape[1], 3, hidden_dim).to(device)
#model.load_state_dict(torch.load('halo_to_galaxy_model3cuantiles.pth'))
#quantiles = np.linspace(0, 1, 50)[1:-1]  # Excluir 0 y 1
quantiles = [0.25, 0.5, 0.75] 
criterion = customLossYan(quantiles)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

In [8]:
print(y_test)

tensor([10.2895,  9.3304,  9.3432,  ...,  9.6249,  9.6279,  9.5978],
       device='cuda:0')


In [9]:
for epoch in range(num_epochs):
    model.train()  
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(targets, outputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    train_loss /= len(train_loader.dataset)     
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(targets, outputs)
            val_loss += loss.item() * inputs.size(0)
    
    val_loss /= len(val_loader.dataset)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'halo_to_galaxy_model3cuantiles.pth')
    else:
        epochs_no_improve += 1
        if epochs_no_improve == early_stop_patience:
            print('Early stopping!')
            break

torch.save(model.state_dict(), 'halo_to_galaxy_model3cuantiles.pth')


Epoch [1/1000], Loss: 0.1912, Val Loss: 0.2089
Epoch [2/1000], Loss: 0.1959, Val Loss: 0.1955
Epoch [3/1000], Loss: 0.1739, Val Loss: 0.2202
Epoch [4/1000], Loss: 0.2777, Val Loss: 0.2454
Epoch [5/1000], Loss: 0.2056, Val Loss: 0.2476
Epoch [6/1000], Loss: 0.1662, Val Loss: 0.1888
Epoch [7/1000], Loss: 0.2858, Val Loss: 0.2623
Epoch [8/1000], Loss: 0.2873, Val Loss: 0.2649
Epoch [9/1000], Loss: 0.2021, Val Loss: 0.1958
Epoch [10/1000], Loss: 0.1875, Val Loss: 0.1987
Epoch [11/1000], Loss: 0.2914, Val Loss: 0.2523
Epoch [12/1000], Loss: 0.1864, Val Loss: 0.1997
Epoch [13/1000], Loss: 0.2048, Val Loss: 0.1908
Epoch [14/1000], Loss: 0.1849, Val Loss: 0.2010
Epoch [15/1000], Loss: 0.2375, Val Loss: 0.2263
Epoch [16/1000], Loss: 0.1744, Val Loss: 0.1900
Epoch [17/1000], Loss: 0.1917, Val Loss: 0.2091
Epoch [18/1000], Loss: 0.1914, Val Loss: 0.2199
Epoch [19/1000], Loss: 0.2018, Val Loss: 0.1972
Epoch [20/1000], Loss: 0.2085, Val Loss: 0.1909
Epoch [21/1000], Loss: 0.2055, Val Loss: 0.2007
E

In [11]:
model.load_state_dict(torch.load('halo_to_galaxy_model3cuantiles.pth'))

model.eval()
test_loss = 0.0
correct = 0
total = 0
'''with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(targets, outputs)
        test_loss += loss.item() * inputs.size(0)
        
        # Calcular precisión
        predicted = outputs.round()  # Redondear las predicciones para compararlas con las etiquetas
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

test_loss /= len(test_loader.dataset)
accuracy = correct / total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}')'''

# KS test
y_test_all = []
outputs_all = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        y_test_all.append(targets.cpu().numpy())
        outputs_all.append(outputs.cpu().numpy())
y_test_all = np.concatenate(y_test_all)
outputs_all = np.concatenate(outputs_all)
ks_statistic, p_value = ks_test_metric(torch.tensor(y_test_all), torch.tensor(outputs_all))
print(f'KS Statistic: {ks_statistic:.4f}, P-value: {p_value:.4f}')

KS Statistic: 1.0000, P-value: 0.0000
